In [ ]:
'''
필요한 라이브러리를 불러옵니다
'''

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('apartment.csv', encoding='cp949') #아파트 데이터를 불러옵니다.

20240101로 되어있는 데이터를 2024:년과 202401:월로 분리합니다.

In [ ]:
df['년'] = df['계약일']//10000
df['년월'] = df['계약일']//100 

2020년부터의 데이터만 사용합니다.

In [ ]:
df = df[df['년'] >= 2020]

In [ ]:
df.info()

결측치가 많거나, 겹치는 변수(자치구명과 자치구코드), 영향을 주지 않는 변수등은 제거하였습니다. 

In [ ]:
df = df[['자치구명', '법정동명', '본번', '부번', '건물명', '계약일', '물건금액(만원)', '건물면적(㎡)', '층', '건축년도', '년', '년월']]

본번과 부번을 이용해서 지번주소를 만듭니다.

In [ ]:
def transform_address(row):
    main_no = str(row['본번'])
    sub_no = (row['부번'])
    sub_no_str = '' if sub_no == 0 else f'-{sub_no}'
    return f"{row['자치구명']} {row['법정동명']} {main_no}{sub_no_str}"

df['주소'] = df.apply(transform_address, axis=1)

건축년도는 연식으로 바꾸기 위해, 건축년도를 매매년에서 빼줍니다.

In [ ]:
df['건축년도'] = df['년'] - df['건축년도']

영어로 columns의 이름을 바꿉니다.

In [ ]:
new_column_names = {'자치구명': 'Gu', '법정동명': 'Dong','건물명': 'Name', '계약일': 'Date', '물건금액(만원)': 'Price', '건물면적(㎡)': 'Area', '층': 'Floor', '건축년도': 'Building Year', '주소' : "Address", '년' : 'Year', '년월': 'Month'}
df2 = df.rename(columns=new_column_names)

In [ ]:
df2.drop(columns=['본번', '부번'], inplace=True)

인구 데이터를 불러옵니다.

In [ ]:
Population = pd.read_csv('Population.csv',encoding='cp949')

0~12: 어린이
13~24 : 청소년
25~40 : 청년
41~65: 중장년
66~ : 노년

으로 매핑합니다.

In [ ]:
population = Population[Population['시도명'] == '서울특별시'] 

population['어린이인구'] = population[[f"{age}세남자" for age in range(13)] + [f"{age}세여자" for age in range(13)]].sum(axis=1)
population['청소년인구'] = population[[f"{age}세남자" for age in range(13, 25)] + [f"{age}세여자" for age in range(13, 25)]].sum(axis=1)
population['청년인구'] = population[[f"{age}세남자" for age in range(25, 41)] + [f"{age}세여자" for age in range(25, 41)]].sum(axis=1)
population['중장년인구'] = population[[f"{age}세남자" for age in range(41, 66)] + [f"{age}세여자" for age in range(41, 66)]].sum(axis=1)
population['노년인구'] = population[[f"{age}세남자" for age in range(66, 109)] + [f"{age}세여자" for age in range(66, 109)]].sum(axis=1)

In [ ]:
population = population[['읍면동명', '계', '어린이인구', '청소년인구', '청년인구', '중장년인구', '노년인구']]
population2 = population.rename(columns={'읍면동명':'Dong', '계':'Total Population', '어린이인구':'Children', '청소년인구':'Adolescent', '청년인구':'Youth', '중장년인구':'Middle-Aged', '노년인구':'Old Age'})

In [ ]:
merged_df = pd.merge(df2, population2, on='Dong', how='left')

인구를 직접 쓰기보다, 비율로 저장합니다.

In [ ]:
merged_df['Children'] = merged_df['Children']/merged_df['Total Population']
merged_df['Youth'] = merged_df['Youth']/merged_df['Total Population']
merged_df['Adolescent'] = merged_df['Adolescent']/merged_df['Total Population']
merged_df['Middle-Aged'] = merged_df['Middle-Aged']/merged_df['Total Population']
merged_df['Old Age'] = merged_df['Old Age']/merged_df['Total Population']

아래는 간단한 데이터 합치는 과정입니다.

In [ ]:
consumption = pd.read_csv('consumption.csv',encoding='utf-8')

In [ ]:
consumption.drop(0, inplace=True)
consumption.drop(columns='경제활동별(1)', inplace=True)
consumption.rename(columns={"자치구(1)":'Gu', '2021':'Consumption'}, inplace=True)

In [ ]:
merged_df2 = pd.merge(merged_df,consumption, how='left', on='Gu')

In [ ]:
merged_df2['Consumption'] = merged_df2['Consumption'].astype(float)/10000
merged_df2['Consumption'] = np.log1p(merged_df2['Consumption'])

In [ ]:
LPI = pd.read_csv('LPI.csv',encoding='utf-8')
API = pd.read_csv('Actual_Price_Index.csv',encoding='utf-8')
interest = pd.read_csv('interest rate.csv', encoding='utf-8')
jeonse = pd.read_csv('jeonse.csv', encoding='cp949')

In [ ]:
LPI['시점'] = (LPI['시점']*100).astype(int)

In [ ]:
LPI.rename(columns={'시점':'Month', '시도별':'Seoul', '총지수':'Price Index','생활물가지수':'Living Price Index'}, inplace=True)

In [ ]:
merged_df3 = pd.merge(merged_df2, LPI, on='Month', how='left')

In [ ]:
merged_df3.drop(columns=['Price Index', 'Seoul'],axis=1,inplace=True)

In [ ]:
API['시점'] = API['시점'].str[:4] + API['시점'].str[5:]

In [ ]:
API['시점'] = API['시점'].astype(str)
API['Month'] = pd.to_numeric(API['시점'], errors='coerce')
merged_df4 = pd.merge(merged_df3, API, on='Month', how='left')

In [ ]:
merged_df4.drop(columns=['항목','시점'], axis=1, inplace=True)

In [ ]:
df4 = merged_df4.rename(columns={'서울': 'Actual Price Index'})

In [ ]:
interest['날짜'] = interest['날짜'].str[:6]
interest['날짜'] = interest['날짜'].astype(int)
merged_df5 = pd.merge(df4, interest, left_on='Month', right_on='날짜', how='left')

In [ ]:
merged_df5.drop('날짜', axis=1, inplace=True)

In [ ]:
df5 = merged_df5.rename(columns={'기준금리': 'Interest Rate'})

네이버 지도 API를 이용해 주소로 위도, 경도를 구해줍니다.

In [ ]:
import requests
from typing import Dict, Union, Optional

NAVER_API_URL: str = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode'
NAVER_API_HEADERS: Dict[str, str] = {
    'X-NCP-APIGW-API-KEY-ID': 'Private_ID',
    'X-NCP-APIGW-API-KEY': 'Private_KEY'
}

def request_naver_api(address: str) -> Dict:
    try:
        response = requests.get(
            NAVER_API_URL,
            params={'query': address},
            headers=NAVER_API_HEADERS
        )
        data: Dict = response.json()
        return data
    except Exception as error:
        raise error

def parse_address_data(data: Dict) -> Union[Dict[str, str], bool]:
    if data['meta']['totalCount'] == 0:
        return False

    Longitude: str = data['addresses'][0]['x']
    Latitude: str = data['addresses'][0]['y']

    return {'Longitude': Longitude, 'Latitude': Latitude}

def get_address_details(add1: str, add2: str) -> Dict[str, str]:
    data: Dict = request_naver_api(add1)
    result1: Union[Dict[str, str], bool] = parse_address_data(data)

    if not result1:
        data = request_naver_api(add2)
        result2: Union[Dict[str, str], bool] = parse_address_data(data)

        if not result2:
            return {'x': "", 'y': ""}

        return result2

    return result1

import pandas as pd

df = pd.read_csv('/content/seoul_apart_price.csv')
#encoding='cp949'

df['address'] = df.apply(lambda row: get_address_details(row['지번주소'], row['도로명주소']), axis=1)

#df을 딕셔너리로 변환
rows_list = df.to_dict('records')

merged_list = []

for row in rows_list:

    address = row.pop('address')
    merged = {**row, **address}
    merged_list.append(merged)

new_df = pd.DataFrame(merged_list)

# 새 DataFrame을 CSV 파일로 저장
new_df.to_csv('/content/seoul_apart_price.csv', index=False)

API를 돌리는데 시간이 소요되어 위도, 경도를 매핑한 다음 파일에서 이어서 하겠습니다.